In [192]:
import numpy as np
import numpy
import torch.nn as nn

import argparse

import os
import numpy as np
import torch
from PIL import Image

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from engine import train_one_epoch, evaluate
import utils
import transforms as T
import time

import os
import numpy as np
import torch
from PIL import Image
import torch.nn as nn

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchsummary import summary

from engine import train_one_epoch, evaluate
import utils
import transforms as T

import matplotlib.pyplot as plt
import matplotlib

import numpy as np
import pandas as pd
from tqdm import tqdm

matplotlib.use('TkAgg')

In [193]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [194]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model.backbone.body.conv1 = nn.Conv2d(9, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [195]:
def load_all_label(path):
    replay_name = path.split('/')[-2]
    print("replay_name:", replay_name)
#     label_path = path.replace(replay_name + "/", '')
    label_len = len(os.listdir(path))
    labels = np.array([])
#     print("label_path:",label_path)
    for i in range(0, label_len):
        labels = np.append(labels, np.load(path +"/"+ str(i) + ".npy", allow_pickle=True)[1])

    results = labels

    return results

In [196]:
class PennFudanDataset(object):
    def __init__(self, path, transforms, window_size):
        self.root = path
        self.transforms = transforms
        pth = os.listdir(path)
        self.label_sequences = []

        self.dir_paths = []
        if os.path.isdir(path):
                self.label_sequences += [load_all_label(path + '/')]
                self.dir_paths.append(path +  '/')
                print(f"Loaded {path}")

        self.window_size = window_size

        self.seq_indexs = []
        start = 0
        for i, seq in enumerate(self.label_sequences):
            end = start + len(seq) - (self.window_size - 1) - 150*(i+1)
            self.seq_indexs.append((i, start, end))
            start = end


    def __len__(self):
        return self.seq_indexs[-1][-1]

    def __getitem__(self, idx):
        # load images and masks
        for i, start, end in self.seq_indexs:
            if idx >= start and idx < end:
                real_idx = idx - start + 150 # *(i+1)

                data = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[0][0]    # (11, 128, 128)
                masks1 = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[1][0]   #  (128, 128)
                masks2 = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[2][0]  # (128, 128)
                masks3 = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[3][0]  # (128, 128)
                masks4 = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[4][0]  # (128, 128)
                masks5 = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[5][0]  # (128, 128)
                masks = np.stack((masks1, masks2,masks3,masks4,masks5))

                break

        input_data = self.preprocessing(data)

        num_objs = 5
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return input_data, target


    def preprocessing(self, data):
        #0 ground 1 air 2 building 3 spell 4 ground 5 air 6 building 7 spell 8 resource 9 vision 10 terrain
        temp = np.zeros([self.window_size, 9, data.shape[2],data.shape[2]])

        temp[:,0] = data[0]
        temp[:,1] = data[1]
        temp[:,2] = data[2]
        temp[:,3] = data[4]

        temp[:,4] = data[6]
        temp[:,5] = data[7]
        temp[:,6] = data[8]
        temp[:,7] = data[10]

        temp[:,8] = data[13]


        data = temp
        data = data.reshape(self.window_size*data.shape[1],data.shape[2],-1)
        # #data = data.reshape(self.window_size*data.shape[0],data.shape[1],-1)
        # label = np.array([label[0]/3456, label[1]/3720])
        return torch.FloatTensor(data)

    # def __len__(self):
    #     return len(self.imgs)


In [197]:
import sys
data_path = [f"./testing_data2/6254/6254", f"./trainig_data2/36/",f"./trainig_data2/212/",f"./trainig_data2/438/",f"./trainig_data2/522/",f"./trainig_data2/1660/"]
for i in data_path:
    print(i.split('/')[-2])

6254
36
212
438
522
1660


In [ ]:
for i in data_path:
    pth = os.listdir(i)    
    replay_name = int(i.split('/')[-2])
    print(i)
    dataset = PennFudanDataset(i, get_transform(train=False), window_size=1)
    
    dataset_len = len(dataset)
    Start, End, Step = 0, len(dataset), 1
    test_img_array = []
    test_img_one_channel_array = []
    test_target_array = []
    
    num_classes = 2
    model = get_model_instance_segmentation(num_classes)
    model.load_state_dict(torch.load("saved_models/model_29.pth"))
    model.eval()
    
    for i in range(Start,End,Step):
        img_t, target_t = dataset[i]
        test_img_array.append(img_t)
        img_one_channel = img_t.sum(axis=0, keepdim=True)
        test_img_one_channel_array.append(img_one_channel)
        test_target_array.append(target_t)
        
    vpx_array = []
    vpy_array = []

    with torch.no_grad():
        for idx,i in tqdm(enumerate(test_img_array)):
            prediction = model(torch.unsqueeze(i, 0))
            if prediction[0]["boxes"].shape[0] == 0:
                prediction = model(torch.unsqueeze(i-1, 0))
                vpx = int(prediction[0]["boxes"][0][1])*32
                vpy = int(prediction[0]["boxes"][0][0])*32  
            else :
                vpx = int(prediction[0]["boxes"][0][1])*32
                vpy = int(prediction[0]["boxes"][0][0])*32
            vpx_array.append(vpx)
            vpy_array.append(vpy)
            
    # dataset_len = 2000
    os.makedirs("./saved_xy/", exist_ok=True)
    temp = np.zeros((dataset_len,1))
    for i in range (0,dataset_len):
        temp[i] = int(i*8)
    temp2 = np.zeros((int(temp.max()),1))
    for i in range(0, int(temp.max())):
        temp2[i] = i

    dataset_temp = pd.DataFrame({"frame": temp[:,0],"vpx": vpx_array[:],"vpy": vpy_array[:]})
    dataset_temp2 = pd.DataFrame({"frame": temp2[:,0]})
    dataset = pd.merge(left = dataset_temp2, right = dataset_temp, how= "left", on = "frame")
    dataset = dataset.fillna(method="ffill")
    dataset.to_csv("./saved_xy/" + str(replay_name) + ".rep.vpd", header= True, index=False)
    dataset.to_csv("C:/TM/starcraft/bwapi-data/read/preset/" + str(replay_name) + ".rep.vpd", header= True, index=False)

    print("saved")

./testing_data2/6254/6254
replay_name: 6254
Loaded ./testing_data2/6254/6254


1131it [53:44,  2.89s/it]